In [167]:
# we could use also tensor flow, there are multiple implementations of word2vec
from gensim.models import word2vec
import pandas as pd
import json
from pathlib import Path
import string
import re

In [168]:
#word to vec exploration  

## .strip(string.punctuation).lower()


Object `re.replace` not found.


In [181]:
#first load all recipe data
BOrecipes = []
BOingredients = []
BOids = []
BOpath = Path('~/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/')
for i, po in enumerate(BOpath.expanduser().glob('*.json')):
    with po.open() as fh:
        try:
            r = json.load(fh)
        except Exception:
            print(str(po))
            continue
        BOrecipes.append(r)
        BOing = r.get('Ingredients')
        BOid = r.get('RecipeID')
        BOids.append(id)
        if BOing is not None:
            #BOingredients.append([i['Name'].strip(string.punctuation).lower().strip(string.digits).strip() for i in BOing if i is not None and i['Name'] is not None])
            BOingredients.append([re.sub(r'[^a-z ]', '', i['Name'].lower().strip()) for i in BOing if i is not None and i['Name'] is not None])

/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-68-87332.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-71-87778.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-19-156227.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-127-74-2431270.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-112-159-862538.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-62-166860.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-35-191-157959.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-76-83346.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-74-82-565669.json
/Users/bridgitboulahanis/Dropbox

In [172]:
#now we want to load spoonacular data and create a dataframe that has ingredients and their aisle
SArecipes = []
SAingredients = []
SAingclass = []
SAnoise = []
SApath = Path('~/Dropbox/Not_work/insight/greenmarkets/spoonac/data/')
for i, po in enumerate(SApath.expanduser().glob('*/*.json')):
    with po.open() as fh:
        try:
            r = json.load(fh)
        except Exception:
            print(str(po))
            continue
        for rec in r:
            currenting = rec['extendedIngredients']
            for ing in currenting:
                SAingredients.append(ing['name'])
                SAingclass.append(ing['aisle'])  
                #SAnoise.extend(ing['metaInformation'])
                SAnoise.extend(i.strip() for i in ing['metaInformation'])

                    





In [173]:
classifying = pd.DataFrame({'ingredient': SAingredients, 'aisle': SAingclass})
classifying.drop_duplicates(inplace=True)

In [174]:
classifying

,ingredient,aisle
0,bell pepper,Produce
1,canned coconut milk,Canned and Jarred
2,carrots,Produce
3,cornstarch,Baking
4,curry powder,Spices and Seasonings
...,...,...
211971,aji amarillo chile paste,Ethnic Foods
211986,braise,?
212025,seasalt,Spices and Seasonings
212028,camembert cheese,Cheese


In [175]:
#some words you noticed should get added to SAnoise
wta = ['ground']
SAnoise.extend(wta)
SAnoise = set(SAnoise)
#remember you can check if it is in there with: 'fresh' in SAnoise

In [176]:
#suite of deduping functions

#on loading we tokenize, strip punctuation&digits&spaces and make lowercase 

def removenoise(input_ing):
    noise_free_ing = []
    for word in input_ing:
        checked = []
        splitit = word.split()
        checked.extend(i for i in splitit if i not in SAnoise)
        noise_free_ing.append(' '.join(checked))
    return noise_free_ing
        
#stem? lemmatize?
    
    

In [182]:
#call the dedupe functions
nning = []
for i in BOingredients:
    nning.append(removenoise(i))



In [183]:
nning[:2]

[['garam masala',
  'coriander leaves',
  'ginger root',
  'turmeric',
  'water',
  'coriander',
  'water',
  'cumin',
  'pepper',
  'rice',
  'tea leaves',
  'chana dal',
  'salt',
  'ghee',
  'cumin seed',
  'onion'],
 ['pasta',
  'shrimp',
  'butter',
  'parsley',
  'pepper flakes',
  'pepper',
  'kosher salt',
  'wine',
  'garlic',
  'parmesan cheese',
  'lemon']]

In [204]:
#classify ingredients
# whichaisle = {}
# for i in nning:
#     for j in i:
#         a = classifying["ingredient"].str.find(j)
#         break
#     break



a = classifying.index[classifying["ingredient"].str.match('butter')]
print(a)
classifying.loc[a]

Int64Index([   275,    800,   2074,   4095,   5408,   7318,  14727,  46865,
             60467,  61573,  64130,  66486,  66537,  70317,  95742, 121487,
            125790, 166839, 181582, 190594],
           dtype='int64')


,ingredient,aisle
275,butter,"Milk, Eggs, Other Dairy"
800,butternut squash,Produce
2074,butter lettuce,Produce
4095,butter nut squash,Produce
5408,buttermilk,"Milk, Eggs, Other Dairy"
7318,butternut pumpkin,Produce
14727,butter lettuce leaves,Produce
46865,butter leaf lettuce,Produce
60467,butter beans,Pasta and Rice;Canned and Jarred
61573,buttermilk biscuits,Refrigerated


In [178]:
#Actual W2V modeling
# Set values for NN parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 3    # 50% of the corpus                    
num_workers = 4       # Number of CPUs
context = 10          # Context window size; 
downsampling = 1e-3   # threshold for configuring which 
                    # higher-frequency words are randomly downsampled

# Initialize and train the model 
model = word2vec.Word2Vec(nning, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

In [164]:
print(model.wv.most_similar('feta cheese', topn=50))
print('=============')
print(model.wv.most_similar('baby spinach'))

[('baby spinach', 0.9684560298919678), ('grape tomatoes', 0.9616087675094604), ('kalamata olives', 0.955957293510437), ('pine nuts', 0.9526952505111694), ('plum tomatoes', 0.9497339725494385), ('basil leaves', 0.944475531578064), ('gemelli pasta', 0.9423145055770874), ('sun-dried tomatoes', 0.942220151424408), ('cherry tomatoes', 0.9412634372711182), ('goat cheese', 0.9357326030731201), ('san marzano tomatoes', 0.9329780340194702), ('balsamic glaze', 0.9199191927909851), ('oil tomatoes', 0.9155914783477783), ('oil cured olives', 0.9134278893470764), ('penne pasta', 0.9080536961555481), ('calamata olives', 0.9077562689781189), ('salt, sea', 0.9041534662246704), ('jar roasted peppers', 0.9012609720230103), ('roma tomatoes', 0.9005147218704224), ('bruschetta', 0.9001086354255676), ('pesto', 0.8996344804763794), ('roasted peppers', 0.8992140293121338), ('anchovy filets', 0.8974788188934326), ('oregano sprigs', 0.8971326351165771), ('zest of 1/2 lemon', 0.8962260484695435), ('extra virgin o

In [89]:
ingredient = 'feta cheese'
df = pd.DataFrame([i for i in ingredients if ingredient in i])
w2v_ingredient = ingredients[1]
w2v_ingredient = 'feta'
print("testing", w2v_ingredient)
similar_ings = [similar_ing for similar_ing, _ in model.wv.most_similar(w2v_ingredient)]
print(similar_ings)
# invers_similar_ings = {}
# for si in similar_ings:
#     for isi, _ in model.wv.most_similar(si):
#         invers_similar_ings.setdefault(isi, 0)
#         invers_similar_ings[isi] += 1
    
# print()
# print("inverse", sorted(invers_similar_ings))

# print("counts")
# for i in sorted(list(invers_similar_ings.items()), key=lambda v: v[1])[-10:]:
#     print(i)

# print()
# print("most similar", model.wv.most_similar(list(invers_similar_ings)))

# print("negative")
# model.wv.most_similar(negative=w2v_ingredient)

testing feta
['egg yolks ', ' Freshly ground black pepper  ', 'fresh garlic ', 'red peppers', 'Coarse salt and freshly ground pepper', 'chorizo sausage', 'fresh lemon', 'butter  ', ' Salt; to taste ', 'green peas']
